# Entraînement du Modèle

Ce notebook entraîne différents modèles de classification pour les panneaux de signalisation.

## Objectifs
- Créer et entraîner un modèle CNN
- Évaluer les performances
- Comparer différents architectures


In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent / "src"))

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from src.data_loader import GTSRBDataLoader
from src.preprocessing import ImagePreprocessor
from src.model import TrafficSignClassifier
from src.utils import plot_training_history, evaluate_model

# Configuration
DATA_PATH = "../data"
MODEL_PATH = "../models"
plt.style.use('seaborn-v0_8')

# Vérifier la disponibilité du GPU
print("GPU disponible:", tf.config.list_physical_devices('GPU'))


## 1. Chargement et Préparation des Données


In [ ]:
# Charger les données
loader = GTSRBDataLoader(DATA_PATH)
X, y = loader.load_train_data(img_size=(64, 64))

print(f"Images chargées: {len(X)}")
print(f"Classes: {len(set(y))}")

# Prétraitement
preprocessor = ImagePreprocessor()
X_train, X_test, y_train, y_test = preprocessor.prepare_data(
    X, y, test_size=0.2, normalize=True
)

print(f"\nTrain: {X_train.shape}")
print(f"Test: {X_test.shape}")


## 2. Création du Modèle CNN


In [ ]:
# Créer le classifieur
num_classes = len(set(y))
classifier = TrafficSignClassifier(num_classes=num_classes)

# Créer le modèle CNN
model = classifier.create_cnn_model()
model.summary()


## 3. Entraînement


In [ ]:
# Entraîner le modèle
history = classifier.train(
    X_train, y_train,
    X_test, y_test,
    epochs=50,
    batch_size=32
)


## 4. Visualisation de l'Historique


In [ ]:
# Afficher l'historique d'entraînement
plot_training_history(history)


## 5. Évaluation du Modèle


In [ ]:
# Évaluer le modèle
class_names = list(range(num_classes))
evaluate_model(model, X_test, y_test, class_names)


## 6. Sauvegarde du Modèle


In [ ]:
# Sauvegarder le modèle
import os
os.makedirs(MODEL_PATH, exist_ok=True)
model_path = f"{MODEL_PATH}/traffic_sign_cnn.h5"
classifier.save_model(model_path)


## 7. Test avec ResNet (Optionnel)

Pour de meilleures performances, vous pouvez essayer ResNet avec transfer learning.


In [ ]:
# Créer un modèle ResNet
resnet_classifier = TrafficSignClassifier(num_classes=num_classes)
resnet_model = resnet_classifier.create_resnet_model()
resnet_model.summary()

# Entraîner (moins d'époques car transfer learning)
resnet_history = resnet_classifier.train(
    X_train, y_train,
    X_test, y_test,
    epochs=20,
    batch_size=32
)

# Sauvegarder
resnet_path = f"{MODEL_PATH}/traffic_sign_resnet.h5"
resnet_classifier.save_model(resnet_path)
